In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
#
print(X_train_full.shape)
print(X_train_full.dtype)

(60000, 28, 28)
uint8


In [3]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [4]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
              "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [5]:
del X_train_full, y_train_full

# The Vanishing/Exploding Gradients Problems
## glorot and He Initialization

In [6]:
# He initialization (for ReLU), normal distribution
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [7]:
# He initialization (for ReLU), uniform distribution
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                                distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

## Nonsaturating Activation Functions

In [8]:
# Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # ReLU, not specifying the initialization method
    keras.layers.Dense(300, activation="relu"),
    # Leaky ReLU, with He-normal initializer
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    # SELU (must use the LeCun normal initializer, sigma^2 = 1/fan_in )
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    #
    keras.layers.Dense(10, activation="softmax")
])

# Batch Normalization

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_8 (Dense)              (None, 10)               

In [11]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [12]:
model.layers[1].updates

/home/benson516/ml/my_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

In [13]:
# Put the batch normalization before activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

# Gradient Clipping

In [14]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

# Reusing Pretrained Layers

In [15]:
import numpy as np

In [16]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [17]:
print(X_train_A.shape)
print(X_train_B.shape)

(43986, 28, 28)
(200, 28, 28)


In [18]:
print(y_train_A[:30])
print(y_train_B[:30])

[4 0 5 7 7 7 4 4 3 4 0 1 6 3 4 3 2 6 5 3 4 5 1 3 4 2 0 6 7 1]
[1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 1. 1.]


In [19]:
# del X_train, y_train, X_valid, y_valid

In [20]:
tf.random.set_seed(42)
np.random.seed(42)

In [21]:
# # Build the model_A
# model_A = keras.models.Sequential()
# model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
# for n_hidden in (300, 100, 50, 50, 50):
#     model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
# model_A.add(keras.layers.Dense(8, activation="softmax"))

In [22]:
# model_A.compile(loss="sparse_categorical_crossentropy",
#                 optimizer=keras.optimizers.SGD(lr=1e-3),
#                 metrics=["accuracy"])

In [23]:
# history = model_A.fit(X_train_A, y_train_A, epochs=20,
#                     validation_data=(X_valid_A, y_valid_A))

In [24]:
# model_A.save("my_model_A.h5")

In [25]:
# Naive solution: use the same model architecture as model_A
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [26]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

/home/benson516/ml/my_env/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [27]:
# history = model_B.fit(X_train_B, y_train_B, epochs=20,
#                       validation_data=(X_valid_B, y_valid_B))

In [28]:
model_B.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_14 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_15 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

## Transfer Learning with Keras

In [29]:
# Transfer the model_A to generate model_B for the second task
# **Note: The layers' weights are shared. 
#         Training one will affect another!!
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # Use "layers" member to access layers
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [30]:
# Clone the model_A before constructing another network
model_A_clone = keras.models.clone_model(model_A) # Clone the "architecture" only
model_A_clone.set_weights(model_A.get_weights()) # Set the cloned model's weight as the original one's weights

In [31]:
# Freeze the model's weights in lower layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
    
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [32]:
# Train a few epochs first
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 70ms/step - loss: 0.1821 - accuracy: 0.9600 - val_loss: 0.1742 - val_accuracy: 0.9777
Epoch 2/4
7/7 [==============================] - 0s 36ms/step - loss: 0.1431 - accuracy: 0.9750 - val_loss: 0.1457 - val_accuracy: 0.9828
Epoch 3/4
7/7 [==============================] - 0s 39ms/step - loss: 0.1186 - accuracy: 0.9850 - val_loss: 0.1260 - val_accuracy: 0.9848
Epoch 4/4
7/7 [==============================] - 0s 37ms/step - loss: 0.1014 - accuracy: 0.9850 - val_loss: 0.1122 - val_accuracy: 0.9858


In [33]:
# Unfreeze the model's weights in lower layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = keras.optimizers.SGD(lr=1e-4) # the default lr is 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

/home/benson516/ml/my_env/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [34]:
# Train the rest epochs
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 75ms/step - loss: 0.0928 - accuracy: 0.9850 - val_loss: 0.1113 - val_accuracy: 0.9858
Epoch 2/16
7/7 [==============================] - 0s 44ms/step - loss: 0.0920 - accuracy: 0.9850 - val_loss: 0.1105 - val_accuracy: 0.9858
Epoch 3/16
7/7 [==============================] - 0s 46ms/step - loss: 0.0911 - accuracy: 0.9850 - val_loss: 0.1097 - val_accuracy: 0.9858
Epoch 4/16
7/7 [==============================] - 0s 47ms/step - loss: 0.0903 - accuracy: 0.9850 - val_loss: 0.1089 - val_accuracy: 0.9858
Epoch 5/16
7/7 [==============================] - 0s 48ms/step - loss: 0.0895 - accuracy: 0.9850 - val_loss: 0.1082 - val_accuracy: 0.9858
Epoch 6/16
7/7 [==============================] - 0s 47ms/step - loss: 0.0887 - accuracy: 0.9850 - val_loss: 0.1074 - val_accuracy: 0.9858
Epoch 7/16
7/7 [==============================] - 0s 48ms/step - loss: 0.0879 - accuracy: 0.9850 - val_loss: 0.1067 - val_accuracy: 0.9858
Epoch 8/16
7/7 [===========

In [35]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 1s 9ms/step - loss: 0.0965 - accuracy: 0.9905


[0.0964755117893219, 0.9904999732971191]

# Faster Optimization

In [36]:
# Build the model_A
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

## Momentum Optimization

In [37]:
optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.9)

In [38]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

1375/1375 [==============================] - 53s 37ms/step - loss: 0.3588 - accuracy: 0.8732 - val_loss: 0.3828 - val_accuracy: 0.8515


## Nesterov Accelerated Gradient

In [39]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

In [40]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

1375/1375 [==============================] - 30s 21ms/step - loss: 0.2613 - accuracy: 0.9089 - val_loss: 0.3071 - val_accuracy: 0.8837


## RMSProp

In [41]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

In [42]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

1375/1375 [==============================] - 33s 22ms/step - loss: 0.3786 - accuracy: 0.8660 - val_loss: 0.2811 - val_accuracy: 0.9008


## Adam and Nadam Optimization

In [43]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

 269/1375 [====>.........................] - ETA: 18s - loss: 0.2922 - accuracy: 0.8931

## Learning Rate Scheduling

In [ ]:
# Power scheduling
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

In [ ]:
# Exponential scheduling
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

In [ ]:
# Exponential scheduling (functional)
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [ ]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A), callbacks=[lr_scheduler])

In [ ]:
# Piecewise constant scheduling
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

In [ ]:
# Performance scheduling
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A), callbacks=[lr_scheduler])

In [ ]:
# tf.keras: keras.optimizers.schedules
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

In [ ]:
# model_A.compile(loss="sparse_categorical_crossentropy",
#                 optimizer=optimizer,
#                 metrics=["accuracy"])
# history = model_A.fit(X_train_A, y_train_A, epochs=1,
#                     validation_data=(X_valid_A, y_valid_A))

# Avoiding Overfitting Through Regularization
## l1 and l2 REgularization

In [ ]:
layer = keras.layers.Dense(100, activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, 
                           activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                    kernel_initializer="glorot_uniform")
])

## Dropout

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=1,
                    validation_data=(X_valid_A, y_valid_A))

## Monte Carlo (MC) Dropout

In [ ]:
y_probas = np.stack([model(X_test_A, training=True) for sample in range(10)])
y_proba = y_probas.mean(axis=0)

## Max-norm Regularization

In [ ]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                  kernel_constraint=keras.constraints.maax_norm(1.0))